In [ ]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import as_strided


from pandas._libs import algos as libalgos
from datetime import datetime

#### understand strides, https://stackoverflow.com/questions/53097952/how-to-understand-numpy-strides-for-layman

In [ ]:
df[0].values.strides[0]

In [ ]:
a = np.arange(1,10).reshape(3,3)
a

In [ ]:
a.strides

#### check rolling_spearman_quick

In [ ]:
np.random.seed(10)
df = pd.DataFrame(np.random.randn(100, 3),columns=['a','b','c'])
df = df.astype(float)


In [ ]:
df

In [ ]:
seqa = df[0].values
seqb = df[1].values
stridea = seqa.strides[0]

In [ ]:
window = 5

In [ ]:
ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])

In [ ]:

stridea = seqa.strides[0]
ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
strideb = seqa.strides[0]
ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
ar = pd.DataFrame(ssa)
br = pd.DataFrame(ssb)
ar = ar.rank(1)
br = br.rank(1)

In [ ]:
corrs = ar.corrwith(br, 1)

In [ ]:
def rolling_spearman_corr(df, window, cols = []):
    idx = df.index
    if len(cols)==2:
        ssa = df[cols[0]].values
        ssb = df[cols[1]].values
        ret = rolling_spearman_quick(ssa, ssb, window)
    elif len(cols)==0:
        return 'TBA'
    else:
        raise ValueError("cols must of list of two or zero elements")
        
    return pd.DataFrame(ret, index=idx, columns=)

In [ ]:
numeric_df = df._get_numeric_data()
cols = numeric_df.columns
idx = cols.copy()
mat = numeric_df.to_numpy(dtype=float, na_value=np.nan, copy=False)

In [ ]:
min_periods =5
correl = libalgos.nancorr_spearman(mat, minp=min_periods)

In [ ]:
def rolling_rank_corr(df, window):
    pass

In [ ]:
stridea = seqa.strides[0]
ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
strideb = seqa.strides[0]
ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
ar = pd.DataFrame(ssa)
br = pd.DataFrame(ssb)
ar = ar.rank(1)
br = br.rank(1)

In [ ]:
window = 5
n = 20
p = 3
x = np.asarray(range(1,n*p+1), np.float64).reshape(n,p)
x = np.random.randn(n, p)
display(x)
x.strides

mat_3d = as_strided(x, shape=(n-window+1, window, p), strides=(x.strides[0], x.strides[0], x.strides[1]))


res = np.array([libalgos.nancorr_spearman(x) for x in mat_3d])

a = np.empty((window-1, p, p,))
a[:] = np.nan

res2 = np.concatenate((a, res))

t = [[i]*p for i in df.index]
flat_list1 = [item for sublist in t for item in sublist]
t2 = [df.columns]*n
flat_list2 = [item for sublist in t2 for item in sublist]
arrays = [flat_list1, flat_list2]
midx = pd.MultiIndex.from_arrays(arrays)

pd.DataFrame(res2.reshape(n*p, p), index=midx, columns=df.columns)

In [ ]:
res2.shape

In [ ]:
res2

In [ ]:
df.columns = ['a','b','c']
df.index = pd.date_range(datetime(2021,1,1), datetime(2021,1,20))

In [ ]:
df.rolling

In [ ]:
img = np.array([[[155,  33, 129],
        [161, 218,   6]],

       [[215, 142, 235],
        [143, 249, 164]],

       [[221,  71, 229],
        [ 56,  91, 120]],

       [[236,   4, 177],
        [171, 105,  40]]])

In [ ]:
img.shape

In [ ]:
img.transpose(2,0,1).shape#.reshape(3,-1)


In [ ]:
array([[155, 161, 215, 143, 221,  56, 236, 171],
       [ 33, 218, 142, 249,  71,  91,   4, 105],
       [129,   6, 235, 164, 229, 120, 177,  40]])

In [ ]:
a = np.empty((3,3,))
a[:] = np.nan

In [ ]:
def rolling_spearman_quick(seqa, seqb, window):
    seqa =seqa.values
    seqb =seqb.values
    stridea = seqa.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    ar = pd.DataFrame(ssa)
    br = pd.DataFrame(ssb)
    ar = ar.rank(1)
    br = br.rank(1)
    corrs = ar.corrwith(br, 1)
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)


def rolling_rank_corr(df, window, y = None, min_periods=1):
    if y is None:
        n, p = df.shape
        x = df.values

        # create rolling 3d array
        mat_3d = as_strided(x, shape=(n-window+1, window, p), strides=(x.strides[0], x.strides[0], x.strides[1]))

        # apply spearman corr
        res = np.array([libalgos.nancorr_spearman(x) for x in mat_3d])

        # produce missing array
        a = np.empty((window-1, p, p,))
        a[:] = np.nan

        # concat
        res2 = np.concatenate((a, res))

        # create index
        t = [[i]*p for i in df.index]
        flat_list1 = [item for sublist in t for item in sublist]
        t2 = [df.columns]*n
        flat_list2 = [item for sublist in t2 for item in sublist]
        arrays = [flat_list1, flat_list2]
        midx = pd.MultiIndex.from_arrays(arrays)

        return pd.DataFrame(res2.reshape(n*p, p), index=midx, columns=df.columns)
    else:
        ret_dict = {}
        for col in df.columns:
            ret_dict[col] = rolling_spearman_quick(df[col], y, window=window)
        return pd.DataFrame(ret_dict, index=df.index)
    

def rollingcorr(df, window, col=None, min_periods=None, method='spearman'):
    if method =='pearson':
        return df.rolling(window).corr(col)
    elif method=='spearman':
        return rolling_rank_corr(df, window, y = col, min_periods=1)
    else:
        raise ValueError(f"method {method} not implemented")

pd.core.frame.DataFrame.rollingcorr=rollingcorr

In [ ]:
df = pd.DataFrame(np.random.randn(20, 3))
df.columns = ['a','b','c']
df.index = pd.date_range(datetime(2021,1,1), datetime(2021,1,20))

In [ ]:
df.rollingcorr?

In [ ]:
df[feature_cols].rollingcorr(df[y])

In [ ]:
n*p x p

In [ ]:
df.rolling(5).corr(None)

In [ ]:
ret1 = df.rolling(5).corr() # nx3 by 3 

In [ ]:
ret1.reset_index().query()

In [ ]:
ret1 = df.rolling(5).corr()

In [ ]:
df.rolling(5).corr(df['c'])

In [ ]:
ret_dict = {}
for col in df.columns:
    ret_dict[col] = rolling_spearman_quick(df[col], df['c'], window=5)

In [ ]:
df.rollingcorr?

In [ ]:
df.rollingcorr(5, col=df['c'])